In [488]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import ast

In [489]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import statsmodels.api as sm

In [490]:
courses = pd.read_csv('course_data.csv')

In [491]:
courses_data = courses[['no', 'co', 'hh', 'ha', 'hs', 'he', 'ci', 'cw', 're', 'la', 'u1', 'u2', 'u3', 'le', 'mw', 'sa', 'pr', 'ra', 'h']]

In [492]:
courses_data

,no,co,hh,ha,hs,he,ci,cw,re,la,u1,u2,u3,le,mw,sa,pr,ra,h
0,1.00,1,0,0,0,0,0,0,1,0,3,2,7,U,['1.001'],[''],"[['18.01', '18.01A']]",5.57,13.33
1,1.000,1,0,0,0,0,0,0,1,0,3,2,7,U,[''],[''],[['18.03']],5.70,9.60
2,1.001,1,0,0,0,0,0,0,0,0,3,2,7,G,['1.00'],[''],"[['18.01', '18.01A']]",5.57,13.33
3,1.005,1,0,0,0,0,0,0,0,0,1,0,2,U,[''],[''],[],6.10,2.70
4,1.008,1,0,0,0,0,0,0,0,0,1,1,1,U,[''],[''],[],6.50,2.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3530,WGS.645,WGS,0,0,0,0,0,0,0,0,3,0,9,G,[''],[''],[],0.00,0.00
3531,WGS.700,WGS,0,0,0,0,0,0,0,0,3,0,9,G,[''],[''],[],0.00,0.00
3532,WGS.S10,WGS,0,0,0,0,0,0,0,0,3,0,9,G,[''],[''],[],5.83,10.13
3533,WGS.UR,WGS,0,0,0,0,0,0,0,0,0,0,0,U,[''],[''],[],0.00,0.00


In [493]:
courses_data['pr'] = courses_data['pr'].apply(lambda x: ast.literal_eval(x))

/Users/ryanwelch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [494]:
courses_data['mw'] = courses_data['mw'].apply(lambda x: ast.literal_eval(x))

/Users/ryanwelch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [495]:
courses_data['sa'] = courses_data['sa'].apply(lambda x: ast.literal_eval(x))

/Users/ryanwelch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [496]:
courses_data['le'] = courses_data['le'].apply(lambda x: 1 if x=='G' else 0)

/Users/ryanwelch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [497]:
courses_data = pd.get_dummies(courses_data, columns=['co'])

# Adjust ratings and hours with 'same as' courses

In [498]:
for i, course in courses_data.iterrows():
    
    if course['ra'] == 0.0 and course['sa'] != ['']:
        for sa_course_name in course['sa']:
            sa_course = courses_data.loc[courses_data['no']==sa_course_name.replace(' ', '')]
            if len(sa_course)==0:
                continue
            if sa_course.iloc[0]['ra'] != 0.0:
                courses_data.loc[i, 'ra'] = sa_course.iloc[0]['ra']
                print(course['no'], sa_course_name.replace(' ', ''))
                break
                
    if course['h'] == 0.0 and course['sa'] != ['']:
        for sa_course_name in course['sa']:
            sa_course = courses_data.loc[courses_data['no']==sa_course_name.replace(' ', '')]
            if len(sa_course)==0:
                continue
            if sa_course.iloc[0]['h'] != 0.0:
                courses_data.loc[i, 'h'] = sa_course.iloc[0]['h']
                print(course['no'], sa_course_name.replace(' ', ''))
                break
    

# Adjust ratings and hours with 'meets with' courses

In [499]:
for i, course in courses_data.iterrows():
    
    if course['ra'] == 0.0 and course['mw'] != ['']:
        for mw_course_name in course['mw']:
            mw_course = courses_data.loc[courses_data['no']==mw_course_name.replace(' ', '')]
            if len(mw_course)==0:
                continue
            if mw_course.iloc[0]['ra'] != 0.0:
                courses_data.loc[i, 'ra'] = mw_course.iloc[0]['ra']
                print(course['no'], mw_course_name.replace(' ', ''))
                break
                
    if course['h'] == 0.0 and course['mw'] != ['']:
        for mw_course_name in course['mw']:
            mw_course = courses_data.loc[courses_data['no']==mw_course_name.replace(' ', '')]
            if len(mw_course)==0:
                continue
            if mw_course.iloc[0]['h'] != 0.0:
                courses_data.loc[i, 'h'] = mw_course.iloc[0]['h']
                print(course['no'], mw_course_name.replace(' ', ''))
                break
    

# Feature Engineering

In [500]:
courses_data.columns

Index(['no', 'hh', 'ha', 'hs', 'he', 'ci', 'cw', 're', 'la', 'u1', 'u2', 'u3',
       'le', 'mw', 'sa', 'pr', 'ra', 'h', 'co_1', 'co_10', 'co_11', 'co_12',
       'co_14', 'co_15', 'co_16', 'co_17', 'co_18', 'co_2', 'co_20', 'co_21',
       'co_21A', 'co_21G', 'co_21H', 'co_21L', 'co_21M', 'co_21W', 'co_22',
       'co_24', 'co_3', 'co_4', 'co_5', 'co_6', 'co_7', 'co_8', 'co_9',
       'co_AS', 'co_CC', 'co_CDO', 'co_CMS', 'co_CSB', 'co_CSE', 'co_EC',
       'co_EM', 'co_ES', 'co_ESD', 'co_HST', 'co_IDS', 'co_MAS', 'co_MS',
       'co_NS', 'co_OR', 'co_RED', 'co_SCM', 'co_SDM', 'co_SP', 'co_STS',
       'co_WGS'],
      dtype='object')

In [501]:
courses_data['has_pr'] = courses_data['pr'].apply(lambda x: 0 if (x==[] or x== ['']) else 1)

In [502]:
#flatten prereqs

In [503]:
i = 20
print(courses_data.iloc[i]['pr'])
print(courses_data.iloc[i]['has_pr'])

[['18.02', '18.02A', '18.022']]
1


In [504]:
def flatten_pr(pre_reqs):
    return [course for pre_req in pre_reqs for course in pre_req]

# flatten_pr([['1', '2'], ['3'], ['4', '5', '6'], ['7'], ['8'], ['9', '10']])

In [505]:
def get_rating(no):
    course = courses_data.loc[courses_data['no']==no]
    if len(course)==0:
        return 0.0
    return course.iloc[0]['ra']

# get_rating('6.1220')

In [506]:
def get_hours(no):
    course = courses_data.loc[courses_data['no']==no]
    if len(course)==0:
        return 0.0
    return course.iloc[0]['h']

# get_hours('6.1220')

In [507]:
def avg_max_pr_rating(course):
    ratings = []
    for pr in flatten_pr(course['pr']):
        pr_ra = get_rating(pr)
        if pr_ra != 0.0:
            ratings.append(pr_ra)
    if len(ratings)==0:
        return 0.0, 0.0
    return sum(ratings)/len(ratings), max(ratings)

course = courses_data.loc[courses_data['no']=='6.1210'].iloc[0]
avg_max_pr_rating(course)

(5.24, 5.65)

In [508]:
def avg_max_pr_hours(course):
    hours = []
    for pr in flatten_pr(course['pr']):
        pr_h = get_hours(pr)
        if pr_h != 0.0:
            hours.append(pr_h)
    if len(hours)==0:
        return 0.0, 0.0
    return sum(hours)/len(hours), max(hours)

course = courses_data.loc[courses_data['no']=='6.1210'].iloc[0]
avg_max_pr_hours(course)

(10.94, 11.29)

In [509]:
courses_data['avg_pr_ra'] = courses_data.apply(lambda x: 0.0 if x['has_pr']==0 else avg_max_pr_rating(x)[0], axis=1)
courses_data['max_pr_ra'] = courses_data.apply(lambda x: 0.0 if x['has_pr']==0 else avg_max_pr_rating(x)[1], axis=1)
courses_data['avg_pr_h'] = courses_data.apply(lambda x: 0.0 if x['has_pr']==0 else avg_max_pr_hours(x)[0], axis=1)
courses_data['max_pr_h'] = courses_data.apply(lambda x: 0.0 if x['has_pr']==0 else avg_max_pr_hours(x)[1], axis=1)

In [510]:
course = courses_data.loc[courses_data['no']=='6.1210'].iloc[0]

In [511]:
print(course['avg_pr_ra'])
print(course['max_pr_ra'])
print(course['avg_pr_h'])
print(course['max_pr_h'])

5.24
5.65
10.94
11.29


# Train Multivariate Regression Model for Hours

In [512]:
courses_data

,no,hh,ha,hs,he,ci,cw,re,la,u1,...,co_SCM,co_SDM,co_SP,co_STS,co_WGS,has_pr,avg_pr_ra,max_pr_ra,avg_pr_h,max_pr_h
0,1.00,0,0,0,0,0,0,1,0,3,...,0,0,0,0,0,1,5.34,5.34,11.92,11.92
1,1.000,0,0,0,0,0,0,1,0,3,...,0,0,0,0,0,1,5.16,5.16,10.13,10.13
2,1.001,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,1,5.34,5.34,11.92,11.92
3,1.005,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0.00,0.00,0.00,0.00
4,1.008,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3530,WGS.645,0,0,0,0,0,0,0,0,3,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00
3531,WGS.700,0,0,0,0,0,0,0,0,3,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00
3532,WGS.S10,0,0,0,0,0,0,0,0,3,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00
3533,WGS.UR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00


In [513]:
#only train on courses that have full hours data
valid_courses = courses_data.loc[courses['h']!=0.0]

In [514]:
valid_courses = valid_courses.drop(
    valid_courses.loc[valid_courses['has_pr']==1].loc[valid_courses['avg_pr_h']==0.0].index.to_list())

In [515]:
valid_courses.columns

Index(['no', 'hh', 'ha', 'hs', 'he', 'ci', 'cw', 're', 'la', 'u1', 'u2', 'u3',
       'le', 'mw', 'sa', 'pr', 'ra', 'h', 'co_1', 'co_10', 'co_11', 'co_12',
       'co_14', 'co_15', 'co_16', 'co_17', 'co_18', 'co_2', 'co_20', 'co_21',
       'co_21A', 'co_21G', 'co_21H', 'co_21L', 'co_21M', 'co_21W', 'co_22',
       'co_24', 'co_3', 'co_4', 'co_5', 'co_6', 'co_7', 'co_8', 'co_9',
       'co_AS', 'co_CC', 'co_CDO', 'co_CMS', 'co_CSB', 'co_CSE', 'co_EC',
       'co_EM', 'co_ES', 'co_ESD', 'co_HST', 'co_IDS', 'co_MAS', 'co_MS',
       'co_NS', 'co_OR', 'co_RED', 'co_SCM', 'co_SDM', 'co_SP', 'co_STS',
       'co_WGS', 'has_pr', 'avg_pr_ra', 'max_pr_ra', 'avg_pr_h', 'max_pr_h'],
      dtype='object')

In [516]:
X = valid_courses.drop(['no', 'ra', 'h', 'pr', 'sa', 'mw'], axis=1)
X

,hh,ha,hs,he,ci,cw,re,la,u1,u2,...,co_SCM,co_SDM,co_SP,co_STS,co_WGS,has_pr,avg_pr_ra,max_pr_ra,avg_pr_h,max_pr_h
0,0,0,0,0,0,0,1,0,3,2,...,0,0,0,0,0,1,5.34,5.34,11.92,11.92
1,0,0,0,0,0,0,1,0,3,2,...,0,0,0,0,0,1,5.16,5.16,10.13,10.13
2,0,0,0,0,0,0,0,0,3,2,...,0,0,0,0,0,1,5.34,5.34,11.92,11.92
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.00,0.00,0.00,0.00
4,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3521,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00
3522,1,0,0,0,0,0,0,0,3,0,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00
3523,1,0,0,0,0,0,0,0,3,0,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00
3524,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00


In [517]:
y = valid_courses['h']
y

0       13.33
1        9.60
2       13.33
3        2.70
4        2.40
        ...  
3521     8.00
3522     8.17
3523    12.00
3524    12.00
3532    10.13
Name: h, Length: 2357, dtype: float64

In [518]:
X = sm.add_constant(X)

In [519]:
model = sm.OLS(y, X).fit()

In [520]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      h   R-squared:                       0.235
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     12.14
Date:                Tue, 05 Dec 2023   Prob (F-statistic):           2.49e-95
Time:                        15:25:38   Log-Likelihood:                -6962.9
No. Observations:                2357   AIC:                         1.404e+04
Df Residuals:                    2298   BIC:                         1.438e+04
Df Model:                          58                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.6931      0.371     20.737      0.0

# Generate Hours Predictions

In [521]:
courses_to_predict = courses_data.loc[courses_data['h']==0.0]

In [522]:
X_new = courses_to_predict.drop(['no', 'ra', 'h', 'pr', 'sa', 'mw'], axis=1)

In [523]:
X_new = sm.add_constant(X_new)

In [524]:
y_pred = model.predict(X_new)

In [525]:
y_pred

17       9.501595
23       9.363517
29      10.187022
36       9.839533
37       8.454791
          ...    
3529    10.542399
3530    10.542399
3531    10.542399
3533     8.558653
3534     8.558653
Length: 1135, dtype: float64

In [526]:
for i, pred in y_pred.iteritems():
    courses_data.loc[i, 'h'] = pred

In [527]:
for i, pred in y_pred.iteritems():
    courses.loc[i, 'h'] = pred

# Train Multivariate Regression Model for Ratings

In [528]:
courses_data

,no,hh,ha,hs,he,ci,cw,re,la,u1,...,co_SCM,co_SDM,co_SP,co_STS,co_WGS,has_pr,avg_pr_ra,max_pr_ra,avg_pr_h,max_pr_h
0,1.00,0,0,0,0,0,0,1,0,3,...,0,0,0,0,0,1,5.34,5.34,11.92,11.92
1,1.000,0,0,0,0,0,0,1,0,3,...,0,0,0,0,0,1,5.16,5.16,10.13,10.13
2,1.001,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,1,5.34,5.34,11.92,11.92
3,1.005,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0.00,0.00,0.00,0.00
4,1.008,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3530,WGS.645,0,0,0,0,0,0,0,0,3,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00
3531,WGS.700,0,0,0,0,0,0,0,0,3,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00
3532,WGS.S10,0,0,0,0,0,0,0,0,3,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00
3533,WGS.UR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00


In [529]:
#only train on courses that have full ratings data
valid_courses = courses_data.loc[courses['ra']!=0.0]

In [530]:
#only train on courses that have data for their pre reqs

valid_courses = valid_courses.drop(
    valid_courses.loc[valid_courses['has_pr']==1].loc[valid_courses['avg_pr_ra']==0.0].index.to_list())

In [531]:
valid_courses.columns

Index(['no', 'hh', 'ha', 'hs', 'he', 'ci', 'cw', 're', 'la', 'u1', 'u2', 'u3',
       'le', 'mw', 'sa', 'pr', 'ra', 'h', 'co_1', 'co_10', 'co_11', 'co_12',
       'co_14', 'co_15', 'co_16', 'co_17', 'co_18', 'co_2', 'co_20', 'co_21',
       'co_21A', 'co_21G', 'co_21H', 'co_21L', 'co_21M', 'co_21W', 'co_22',
       'co_24', 'co_3', 'co_4', 'co_5', 'co_6', 'co_7', 'co_8', 'co_9',
       'co_AS', 'co_CC', 'co_CDO', 'co_CMS', 'co_CSB', 'co_CSE', 'co_EC',
       'co_EM', 'co_ES', 'co_ESD', 'co_HST', 'co_IDS', 'co_MAS', 'co_MS',
       'co_NS', 'co_OR', 'co_RED', 'co_SCM', 'co_SDM', 'co_SP', 'co_STS',
       'co_WGS', 'has_pr', 'avg_pr_ra', 'max_pr_ra', 'avg_pr_h', 'max_pr_h'],
      dtype='object')

In [532]:
X = valid_courses.drop(['no', 'ra', 'pr', 'sa', 'mw'], axis=1)
X

,hh,ha,hs,he,ci,cw,re,la,u1,u2,...,co_SCM,co_SDM,co_SP,co_STS,co_WGS,has_pr,avg_pr_ra,max_pr_ra,avg_pr_h,max_pr_h
0,0,0,0,0,0,0,1,0,3,2,...,0,0,0,0,0,1,5.34,5.34,11.92,11.92
1,0,0,0,0,0,0,1,0,3,2,...,0,0,0,0,0,1,5.16,5.16,10.13,10.13
2,0,0,0,0,0,0,0,0,3,2,...,0,0,0,0,0,1,5.34,5.34,11.92,11.92
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.00,0.00,0.00,0.00
4,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3521,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00
3522,1,0,0,0,0,0,0,0,3,0,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00
3523,1,0,0,0,0,0,0,0,3,0,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00
3524,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,1,0,0.00,0.00,0.00,0.00


In [533]:
Y = valid_courses['ra']
Y

0       5.57
1       5.70
2       5.57
3       6.10
4       6.50
        ... 
3521    4.80
3522    6.45
3523    6.00
3524    6.00
3532    5.83
Name: ra, Length: 2360, dtype: float64

In [534]:
X = sm.add_constant(X)

In [535]:
model = sm.OLS(Y, X).fit()

In [536]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                     ra   R-squared:                       0.168
Model:                            OLS   Adj. R-squared:                  0.147
Method:                 Least Squares   F-statistic:                     7.868
Date:                Tue, 05 Dec 2023   Prob (F-statistic):           2.11e-57
Time:                        15:25:38   Log-Likelihood:                -2192.0
No. Observations:                2360   AIC:                             4504.
Df Residuals:                    2300   BIC:                             4850.
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.9732      0.053    112.397      0.0

# Generate Rating Predictions

In [537]:
courses_to_predict = courses_data.loc[courses_data['ra']==0.0]

In [538]:
X_new = courses_to_predict.drop(['no', 'ra', 'pr', 'sa', 'mw'], axis=1)

In [539]:
X_new = sm.add_constant(X_new)

In [540]:
y_pred = model.predict(X_new)

In [541]:
for i, pred in y_pred.iteritems():
    courses_data.loc[i, 'ra'] = pred

In [542]:
for i, pred in y_pred.iteritems():
    courses.loc[i, 'ra'] = pred

# Write to CSV

In [554]:
courses.to_csv('course_data_predicted.csv', index=False)

In [556]:
test = pd.read_csv('course_data_predicted.csv')

In [557]:
np.isclose(courses['ra'], test['ra'], atol=0.00001).all()

True

In [558]:
np.isclose(courses['h'], test['h'], atol=0.00001).all()

True